# Wheelchair Jerk Analysis (Diff-Drive / Unicycle Approximation)

This notebook computes **wheelchair linear jerk** (m/s³) from **left/right wheel encoder speeds**.

**Given:**
- Wheel radius: **r = 0.1325 m**
- Wheel separation (track width): **b = 0.52 m**
- Input CSVs contain:
  - time column (e.g., `t_ref_s`)
  - wheel angles (rad) and speeds (rad/s), including filtered variants like `wheel_left_speed_f5`.

**Outputs:**
- Per-trial time series CSV: `outputs/<trial_name>_jerk_timeseries.csv`
- Per-trial plot PNG: `outputs/<trial_name>_jerk_plot.png`
- Per-trial metrics CSV: `outputs/trial_jerk_metrics.csv`
- Task-level summary (mean±sd across trials): `outputs/task_jerk_summary.csv`


In [ ]:
# If running in Colab and SciPy is missing, uncomment:
# !pip -q install scipy

import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.signal import savgol_filter


In [ ]:
# -----------------------------
# Parameters (edit if needed)
# -----------------------------
R_WHEEL = 0.1325   # wheel radius [m]
B_TRACK = 0.52     # distance between wheels [m]

# Prefer filtered wheel columns if present:
PREFERRED_LEFT_SPEED_COLS  = ["wheel_left_speed_f5", "wheel_left_speed", "left_wheel_speed", "wheel_l_speed", "wl_speed"]
PREFERRED_RIGHT_SPEED_COLS = ["wheel_right_speed_f5", "wheel_right_speed", "right_wheel_speed", "wheel_r_speed", "wr_speed"]

PREFERRED_TIME_COLS = ["t_ref_s", "timestamp", "time", "t", "sec", "t_s"]

OUT_DIR = "outputs"
os.makedirs(OUT_DIR, exist_ok=True)


In [ ]:
def pick_column(df: pd.DataFrame, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    # fuzzy fallback
    cols = list(df.columns)
    low = [x.lower() for x in cols]
    for cand in candidates:
        cl = cand.lower()
        if cl in low:
            return cols[low.index(cl)]
    return None

def ensure_strictly_increasing(t, *arrays):
    t = np.asarray(t, dtype=np.float64)
    keep = np.ones_like(t, dtype=bool)
    keep[1:] = np.diff(t) > 0
    t2 = t[keep]
    out = [np.asarray(a)[keep] for a in arrays]
    return (t2, *out)

def auto_savgol_window(t, target_window_s=0.25, min_win=7, max_win=101):
    """Choose an odd Savitzky–Golay window length based on median dt."""
    t = np.asarray(t, dtype=np.float64)
    dt = np.median(np.diff(t))
    if not np.isfinite(dt) or dt <= 0:
        return min_win
    win = int(round(target_window_s / dt))
    win = max(min_win, min(max_win, win))
    if win % 2 == 0:
        win += 1
    # ensure <= N and odd
    if win >= len(t):
        win = len(t) - 1 if (len(t)-1) % 2 == 1 else len(t) - 2
        win = max(win, min_win)
    return win

def compute_v_omega(left_w, right_w, r=R_WHEEL, b=B_TRACK):
    # left_w/right_w are wheel angular speeds [rad/s]
    v = 0.5 * r * (right_w + left_w)      # m/s
    omega = (r / b) * (right_w - left_w)  # rad/s
    return v, omega

def compute_jerk_from_v(t, v, smooth=True):
    """Return (v_smooth, a, j) with optional smoothing."""
    t = np.asarray(t, dtype=np.float64)
    v = np.asarray(v, dtype=np.float64)

    if smooth:
        win = auto_savgol_window(t, target_window_s=0.25)
        v_s = savgol_filter(v, window_length=win, polyorder=3)
    else:
        v_s = v

    a = np.gradient(v_s, t)   # m/s^2
    j = np.gradient(a, t)     # m/s^3
    return v_s, a, j

def trial_metrics(t, v, j):
    duration = float(t[-1] - t[0]) if len(t) >= 2 else float("nan")
    mean_abs_j = float(np.mean(np.abs(j)))
    rms_j = float(np.sqrt(np.mean(j**2)))
    max_abs_j = float(np.max(np.abs(j)))
    # also report mean speed magnitude as a sanity check
    mean_abs_v = float(np.mean(np.abs(v)))
    return {
        "duration_s": duration,
        "mean_abs_v_mps": mean_abs_v,
        "mean_abs_jerk_mps3": mean_abs_j,
        "rms_jerk_mps3": rms_j,
        "max_abs_jerk_mps3": max_abs_j,
    }


In [ ]:
# -----------------------------
# Select trial CSVs
# -----------------------------
# Option A: put all trial CSVs in the same folder as this notebook and use:
trial_csvs = sorted(glob.glob("*.csv"))

# Option B: specify exact paths:
trial_csvs = ["1.csv",
    "2.csv",
    "3.csv",
    "4.csv",
    ]




If you only see one file here, upload/copy the rest of your trial CSVs into the same folder, then re-run this cell.


In [ ]:
all_metrics = []
timeseries_paths = []
plot_paths = []

for csv_path in trial_csvs:
    name = os.path.splitext(os.path.basename(csv_path))[0]
    df = pd.read_csv(csv_path)

    t_col = pick_column(df, PREFERRED_TIME_COLS)
    l_col = pick_column(df, PREFERRED_LEFT_SPEED_COLS)
    r_col = pick_column(df, PREFERRED_RIGHT_SPEED_COLS)

    if t_col is None or l_col is None or r_col is None:
        print(f"[SKIP] {csv_path}: missing required columns. Found t={t_col}, left={l_col}, right={r_col}")
        continue

    # drop NaNs on the required columns
    d = df[[t_col, l_col, r_col]].copy()
    d = d.dropna()

    t = d[t_col].to_numpy(np.float64)
    wl = d[l_col].to_numpy(np.float64)
    wr = d[r_col].to_numpy(np.float64)

    # ensure increasing time
    t, wl, wr = ensure_strictly_increasing(t, wl, wr)
    if len(t) < 10:
        print(f"[SKIP] {csv_path}: too few samples after cleaning.")
        continue

    v, omega = compute_v_omega(wl, wr)
    v_s, a, j = compute_jerk_from_v(t, v, smooth=True)

    # save time series
    out_ts = os.path.join(OUT_DIR, f"{name}_jerk_timeseries.csv")
    out_df = pd.DataFrame({
        "t_s": t,
        "wheel_left_speed_rad_s": wl,
        "wheel_right_speed_rad_s": wr,
        "v_mps_raw": v,
        "v_mps": v_s,
        "a_mps2": a,
        "jerk_mps3": j,
        "omega_rad_s": omega,
    })
    out_df.to_csv(out_ts, index=False)
    timeseries_paths.append(out_ts)

    # compute metrics
    m = trial_metrics(t, v_s, j)
    m["trial"] = name
    m["csv_path"] = csv_path
    all_metrics.append(m)

    # plot
    out_png = os.path.join(OUT_DIR, f"{name}_jerk_plot.png")
    fig = plt.figure(figsize=(10, 6))
    ax1 = plt.gca()
    ax1.plot(t - t[0], v_s, label="v (smoothed) [m/s]")
    ax1.set_xlabel("time since start [s]")
    ax1.set_ylabel("v [m/s]")
    ax1.grid(True)

    ax2 = ax1.twinx()
    ax2.plot(t - t[0], j, label="jerk [m/s^3]")
    ax2.set_ylabel("jerk [m/s^3]")

    # combine legends
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc="best")

    plt.title(f"{name}: Wheelchair Linear Velocity & Jerk")
    plt.tight_layout()
    plt.savefig(out_png, dpi=200)
    plt.close(fig)
    plot_paths.append(out_png)

len(all_metrics), timeseries_paths[:2], plot_paths[:2]


(4,
 ['outputs/1_jerk_timeseries.csv', 'outputs/2_jerk_timeseries.csv'],
 ['outputs/1_jerk_plot.png', 'outputs/2_jerk_plot.png'])

In [ ]:
# -----------------------------
# Trial-level summary table
# -----------------------------
metrics_df = pd.DataFrame(all_metrics)
metrics_df = metrics_df.sort_values("trial").reset_index(drop=True)

trial_metrics_csv = os.path.join(OUT_DIR, "wheelchair_trial_jerk_metrics.csv")
metrics_df.to_csv(trial_metrics_csv, index=False)

metrics_df


,duration_s,mean_abs_v_mps,mean_abs_jerk_mps3,rms_jerk_mps3,max_abs_jerk_mps3,trial,csv_path
0,208.500000,0.0,0.0,0.0,0.0,1,1.csv
1,179.416667,0.0,0.0,0.0,0.0,2,2.csv
2,120.666667,0.0,0.0,0.0,0.0,3,3.csv
3,166.333333,0.0,0.0,0.0,0.0,4,4.csv


In [ ]:
# -----------------------------
# Task-level summary (mean ± sd across trials)
# -----------------------------
# (If you have multiple tasks, you can run this notebook per-task folder,
#  or extend it by grouping trials by a naming rule.)

if len(metrics_df) == 0:
    raise RuntimeError("No valid trials processed. Check column names and CSV list.")

summary = {}
for col in ["duration_s", "mean_abs_v_mps", "mean_abs_jerk_mps3", "rms_jerk_mps3", "max_abs_jerk_mps3"]:
    vals = metrics_df[col].to_numpy(np.float64)
    summary[col + "_mean"] = float(np.mean(vals))
    summary[col + "_sd"]   = float(np.std(vals, ddof=1)) if len(vals) > 1 else 0.0

task_summary_df = pd.DataFrame([summary])
task_summary_csv = os.path.join(OUT_DIR, "wheelchair_task_jerk_summary.csv")
task_summary_df.to_csv(task_summary_csv, index=False)

task_summary_df


,duration_s_mean,duration_s_sd,mean_abs_v_mps_mean,mean_abs_v_mps_sd,mean_abs_jerk_mps3_mean,mean_abs_jerk_mps3_sd,rms_jerk_mps3_mean,rms_jerk_mps3_sd,max_abs_jerk_mps3_mean,max_abs_jerk_mps3_sd
0,168.729167,36.568136,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Notes / knobs to tune

- If jerk looks too noisy, increase the smoothing window in `auto_savgol_window()` by raising `target_window_s` (e.g., 0.35–0.5 s).
- If you want **yaw jerk** (turning jerk), do the same pipeline on `omega(t)`:
  - `alpha = d omega / dt`, `j_omega = d alpha / dt`.
- Make sure your wheel speeds are truly **rad/s**. If they are RPM, convert first.
